# Alignment and tree building using Nextstrain

* __Notebook version__: `v0.0.1`
* __Created by:__ [Dr. Hiren Ghosh](https://www.linkedin.com/in/hiren-ghosh-phd-6181853a), `Imperial BRC Genomics Facility`
* __Maintained by:__ `Imperial BRC Genomics Facility`
* __Docker image:__ `TODO`
* __Github repository:__ [imperial-genomics-facility/viral-genome-notebook-image](https://github.com/imperial-genomics-facility/viral-genome-notebook-image)
* __Created on:__ `2020-April-21 14:42`
* __Contact us:__ [Imperial BRC Genomics Facility](https://www.imperial.ac.uk/medicine/research-and-impact/facilities/genomics-facility/contact/)
* __License:__ [Apache License 2.0](https://github.com/imperial-genomics-facility/scanpy-notebook-image/blob/master/LICENSE)

In [1]:
import os
import requests
import json
import time
from ete3 import Tree, TreeStyle,SeqMotifFace
os.environ['QT_QPA_PLATFORM']='offscreen'

In [2]:
def fetch_genome_fasta_from_ncbi(refseq_id,output_path='.',file_format='fasta'):
  '''
  A function for fetching the genome fasta sequences from NCBI
  
  :param refseq_id: NCBI genome id
  :param output_path: Path to dump genome files, default '.'
  '''
  try:
    url = \
      'https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=nucleotide&id={0}&rettype={1}'.\
        format(refseq_id,file_format)
    r = requests.get(url)
    fasta_data = r.content.decode('utf-8')
    output_file = \
      os.path.join(
        os.path.abspath(output_path),
        '{0}.{1}'.format(refseq_id,file_format))
    with open(output_file,'w') as fp:
      fp.write(fasta_data)
    print('Downloaded genome seq for {0}'.format(refseq_id))
  except Exception as e:
    print('Failed to download data for {0} from NCBI, error: {1}'.format(refseq_id,e))

In [3]:
genome_list = [
    'MN988668.1',
    'NC_045512.2',
    'MN938384.1',
    'MN975262.1',
    'MN985325.1',
    'MN988713.1',
    'MN994467.1',
    'MN994468.1',
    'MN997409.1'
]

In [4]:
for id in genome_list:
    fetch_genome_fasta_from_ncbi(id)
    time.sleep(2)

Downloaded genome seq for MN988668.1
Downloaded genome seq for NC_045512.2
Downloaded genome seq for MN938384.1
Downloaded genome seq for MN975262.1
Downloaded genome seq for MN985325.1
Downloaded genome seq for MN988713.1
Downloaded genome seq for MN994467.1
Downloaded genome seq for MN994468.1
Downloaded genome seq for MN997409.1


In [5]:
fetch_genome_fasta_from_ncbi('NC_045512.2',file_format='gb')

Downloaded genome seq for NC_045512.2


In [6]:
%%bash
cat MN*.fasta > corona.fa
grep '>' corona.fa|wc -l

8


In [7]:
!augur align \
  --sequences /home/vmuser/examples/corona.fa \
  --reference-sequence /home/vmuser/examples/NC_045512.2.gb \
  --nthreads 1 \
  --output /home/vmuser/examples/corona.afa \
  --fill-gaps


using mafft to align via:
	mafft --reorder --anysymbol --nomemsave --adjustdirection --thread 1 /home/vmuser/examples/corona.afa.to_align.fasta 1> /home/vmuser/examples/corona.afa 2> /home/vmuser/examples/corona.afa.log 

	Katoh et al, Nucleic Acid Research, vol 30, issue 14
	https://doi.org/10.1093%2Fnar%2Fgkf436

No gaps in alignment to trim (with respect to the reference, NC_045512.2)


In [8]:
!augur tree \
  --alignment /home/vmuser/examples/corona.afa \
  --method fasttree \
  --output /home/vmuser/examples/corona.tree \
  --nthreads 1

Cannot specify model unless using IQTree. Model specification ignored.
Building a tree via:
	FastTreeMP -nosupport -nt /home/vmuser/examples/corona.afa  1> /home/vmuser/examples/corona.tree 2> /home/vmuser/examples/corona.tree.log
	Price et al: FastTree 2 - Approximately Maximum-Likelihood Trees for Large Alignments.
	PLoS ONE 5(3): e9490. https://doi.org/10.1371/journal.pone.0009490


Building original tree took 2.335139513015747 seconds


In [9]:
metadata = [{
  'strain':'NC_045512.2',
  'virus':'SARS-CoV-2',
  'accession':'NC_045512',
  'date':'2020-03-30',
  'region':'China',
  'country':'China',
  'division':'China',
  'city':'Wuhan',
  'db':'genbank',
  'segment':'genome',
  'authors':'Baranov,P.V., Henderson,C.M., Anderson,C.B., Gesteland,R.F.,Atkins,J.F. and Howard,M.T.',
  'url':'https://www.ncbi.nlm.nih.gov/nuccore/NC_045512.2',
  'title':'Programmed ribosomal frameshifting in decoding the SARS-CoV genome',
  'journal':'Virology 332 (2), 498-510 (2005)',
  'paper_url':'https://www.ncbi.nlm.nih.gov/pubmed/15680415'
  },
  {
  'strain':'MN988668.1',
  'virus':'SARS-CoV-2',
  'accession':'MN988668',
  'date':'2020-03-11',
  'region':'China',
  'country':'China',
  'division':'China',
  'city':'Wuhan',
  'db':'genbank',
  'segment':'genome',
  'authors':'Chen,L., Liu,W., Zhang,Q., Xu,K., Ye,G., Wu,W., Sun,Z., Liu,F.,Wu,K., Mei,Y., Zhang,W., Chen,Y., Li,Y., Shi,M., Lan,K. and Liu,Y.',
  'url':'https://www.ncbi.nlm.nih.gov/nuccore/MN988668.1',
  'title':'RNA based mNGS approach identifies a novel human coronavirus from two individual pneumonia cases in 2019 Wuhan outbreak',
  'journal':'Emerg Microbes Infect (2019) In press',
  'paper_url':''
  },
  {
  'strain':'MN938384.1',
  'virus':'SARS-CoV-2',
  'accession':'MN938384',
  'date':'2020-02-11',
  'region':'China',
  'country':'China',
  'division':'China',
  'city':'Shenzhen',
  'db':'genbank',
  'segment':'genome',
  'authors':"""Chan,J.F.-W., Yuan,S., Kok,K.H., To,K.K.-W., Chu,H., Yang,J.,
            Xing,F., Liu,J., Yip,C.C.-Y., Poon,R.W.-S., Tsai,H.W., Lo,S.K.-F.,
            Chan,K.H., Poon,V.K.-M., Chan,W.M., Ip,J.D., Cai,J.P.,
            Cheng,V.C.-C., Chen,H., Hui,C.K.-M. and Yuen,K.Y.""",
  'url':'https://www.ncbi.nlm.nih.gov/nuccore/MN938384.1',
  'title':'A familial cluster of pneumonia associated with the 2019 novel coronavirus indicating person-to-person transmission: a study of a family cluster',
  'journal':'Lancet (2020) In press',
  'paper_url':''
  },
  {
  'strain':'MN975262.1',
  'virus':'SARS-CoV-2',
  'accession':'MN975262',
  'date':'2020-02-11',
  'region':'China',
  'country':'China',
  'division':'China',
  'city':'Shenzhen',
  'db':'genbank',
  'segment':'genome',
  'authors':"""Chan,J.F.-W., Yuan,S., Kok,K.H., To,K.K.-W., Chu,H., Yang,J.,
            Xing,F., Liu,J., Yip,C.C.-Y., Poon,R.W.-S., Tsai,H.W., Lo,S.K.-F.,
            Chan,K.H., Poon,V.K.-M., Chan,W.M., Ip,J.D., Cai,J.P.,
            Cheng,V.C.-C., Chen,H., Hui,C.K.-M. and Yuen,K.Y.""",
  'url':'https://www.ncbi.nlm.nih.gov/nuccore/MN975262.1',
  'title':"""A familial cluster of pneumonia associated with the 2019 novel
            coronavirus indicating person-to-person transmission: a study of a
            family cluster""",
  'journal':'Lancet (2020) In press',
  'paper_url':''
  },
  {
  'strain':'MN985325.1',
  'virus':'SARS-CoV-2',
  'accession':'MN985325',
  'date':'2020-03-27',
  'region':'USA',
  'country':'USA',
  'division':'USA',
  'city':'Atlanta',
  'db':'genbank',
  'segment':'genome',
  'authors':"""Harcourt,J., Tamin,A., Lu,X., Kamili,S., Sakthivel,S.K., Murray,J.,
            Queen,K., Tao,Y., Paden,C.R., Zhang,J., Li,Y., Uehara,A., Wang,H.,
            Goldsmith,C., Bullock,H.A., Wang,L., Whitaker,B., Lynch,B.,
            Gautam,R., Schindewolf,C., Lokugamage,K.G., Scharton,D.,
            Plante,J.A., Mirchandani,D., Widen,S.G., Narayanan,K., Makino,S.,
            Ksiazek,T.G., Plante,K.S., Weaver,S.C., Lindstrom,S., Tong,S.,
            Menachery,V.D. and Thornburg,N.J.""",
  'url':'https://www.ncbi.nlm.nih.gov/nuccore/MN985325.1',
  'title':"""Severe Acute Respiratory Syndrome Coronavirus 2 from Patient with
            2019 Novel Coronavirus Disease, United States""",
  'journal':'Emerging Infect. Dis. 26 (6) (2020) In press',
  'paper_url':''
  },
  {
  'strain':'MN988713.1',
  'virus':'SARS-CoV-2',
  'accession':'MN988713',
  'date':'2020-02-11',
  'region':'USA',
  'country':'USA',
  'division':'USA',
  'city':'Illinois',
  'db':'genbank',
  'segment':'genome',
  'authors':"""Tao,Y., Queen,K., Paden,C.R., Zhang,J., Li,Y., Uehara,A., Lu,X.,
            Lynch,B., Sakthivel,S.K.K., Whitaker,B.L., Kamili,S., Wang,L.,
            Murray,J.R., Gerber,S.I., Lindstrom,S. and Tong,S.""",
  'url':'https://www.ncbi.nlm.nih.gov/nuccore/MN988713.1',
  'title':'Novel betacoronavirus, Illinois',
  'journal':'',
  'paper_url':''
  },
  {
  'strain':'MN994467.1',
  'virus':'SARS-CoV-2',
  'accession':'MN994467',
  'date':'2020-02-11',
  'region':'USA',
  'country':'USA',
  'division':'USA',
  'city':'California',
  'db':'genbank',
  'segment':'genome',
  'authors':"""Uehara,A., Queen,K., Tao,Y., Li,Y., Paden,C.R., Zhang,J., Lu,X.,
            Lynch,B., Sakthivel,S.K.K., Whitaker,B.L., Kamili,S., Wang,L.,
            Murray,J.R., Gerber,S.I., Lindstrom,S. and Tong,S.""",
  'url':'https://www.ncbi.nlm.nih.gov/nuccore/MN994467.1',
  'title':'nCoV-2019 California case 1',
  'journal':'',
  'paper_url':''
  },
  {
  'strain':'MN994468.1',
  'virus':'SARS-CoV-2',
  'accession':'MN994468',
  'date':'2020-02-11',
  'region':'USA',
  'country':'USA',
  'division':'USA',
  'city':'California',
  'db':'genbank',
  'segment':'genome',
  'authors':"""Uehara,A., Queen,K., Tao,Y., Li,Y., Paden,C.R., Zhang,J., Lu,X.,
            Lynch,B., Sakthivel,S.K.K., Whitaker,B.L., Kamili,S., Wang,L.,
            Murray,J.R., Gerber,S.I., Lindstrom,S. and Tong,S.""",
  'url':'https://www.ncbi.nlm.nih.gov/nuccore/MN994468.1',
  'title':'nCoV-2019 California case 2',
  'journal':'',
  'paper_url':''
  },
  {
  'strain':'MN997409.1',
  'virus':'SARS-CoV-2',
  'accession':'MN997409',
  'date':'2020-02-11',
  'region':'USA',
  'country':'USA',
  'division':'USA',
  'city':'Arizona',
  'db':'genbank',
  'segment':'genome',
  'authors':"""Tao,Y., Paden,C.R., Queen,K., Uehara,A., Li,Y., Zhang,J., Lu,X.,
            Lynch,B., Sakthivel,S.K.K., Whitaker,B.L., Kamili,S., Wang,L.,
            Murray,J.R., Gerber,S.I., Lindstrom,S. and Tong,S.""",
  'url':'https://www.ncbi.nlm.nih.gov/nuccore/MN997409.1',
  'title':'nCoV-2019 sequence from Arizona case',
  'journal':'',
  'paper_url':''
  }
]


In [10]:
import pandas as pd

In [11]:
df = pd.DataFrame(metadata)

In [12]:
df.to_csv('metadata.tsv',sep='\t',index=False)

In [13]:
!head -2 metadata.tsv

strain	virus	accession	date	region	country	division	city	db	segment	authors	url	title	journal	paper_url
NC_045512.2	SARS-CoV-2	NC_045512	2020-03-30	China	China	China	Wuhan	genbank	genome	Baranov,P.V., Henderson,C.M., Anderson,C.B., Gesteland,R.F.,Atkins,J.F. and Howard,M.T.	https://www.ncbi.nlm.nih.gov/nuccore/NC_045512.2	Programmed ribosomal frameshifting in decoding the SARS-CoV genome	Virology 332 (2), 498-510 (2005)	https://www.ncbi.nlm.nih.gov/pubmed/15680415


In [14]:
!augur refine \
  --tree /home/vmuser/examples/corona.tree \
  --alignment /home/vmuser/examples/corona.afa \
  --metadata /home/vmuser/examples/metadata.tsv \
  --output-tree /home/vmuser/examples/corona.nwk \
  --output-node-data /home/vmuser/examples/corona_branch_lengths.json \
  --timetree \
  --coalescent opt \
  --date-confidence \
  --date-inference marginal \
  --clock-filter-iqd 4


1.01	TreeTime.reroot: with method or node: least-squares

1.01	TreeTime.reroot: rerooting will ignore covariance and shared ancestry.

1.05	TreeTime.reroot: with method or node: least-squares

1.05	TreeTime.reroot: rerooting will ignore covariance and shared ancestry.

1.08	WARNING: Previous versions of TreeTime (<0.7.0) RECONSTRUCTED sequences of
    	tips at positions with AMBIGUOUS bases. This resulted in unexpected
    	behavior is some cases and is no longer done by default. If you want to
    	replace those ambiguous sites with their most likely state, rerun with
    	`reconstruct_tip_states=True` or `--reconstruct-tip-states`.

1.25	TreeTime.reroot: with method or node: least-squares

1.25	TreeTime.reroot: rerooting will account for covariance and shared ancestry.

1.37	###TreeTime.run: INITIAL ROUND

2.00	TreeTime.reroot: with method or node: least-squares

2.00	TreeTime.reroot: rerooting will account for covariance and shared ancestry.

2.07	###TreeTime.run: rerunning timetre

In [15]:
## Annotate the Phylogeny
### Reconstruct Ancestral Traits
!augur traits \
  --tree /home/vmuser/examples/corona.nwk \
  --metadata /home/vmuser/examples/metadata.tsv \
  --output /home/vmuser/examples/corona_traits.json \
  --columns region country \
  --confidence

Assigned discrete traits to 9 out of 9 taxa.

NOTE: previous versions (<0.7.0) of this command made a 'short-branch
length assumption. TreeTime now optimizes the overall rate numerically
and thus allows for long branches along which multiple changes
accumulated. This is expected to affect estimates of the overall rate
while leaving the relative rates mostly unchanged.
Assigned discrete traits to 9 out of 9 taxa.

NOTE: previous versions (<0.7.0) of this command made a 'short-branch
length assumption. TreeTime now optimizes the overall rate numerically
and thus allows for long branches along which multiple changes
accumulated. This is expected to affect estimates of the overall rate
while leaving the relative rates mostly unchanged.

Inferred ancestral states of discrete character using TreeTime:
	Sagulenko et al. TreeTime: Maximum-likelihood phylodynamic analysis
	Virus Evolution, vol 4, https://academic.oup.com/ve/article/4/1/vex042/4794731

results written to /home/vmuser/examples/co

In [16]:
### Infer Ancestral Sequences
!augur ancestral \
  --tree /home/vmuser/examples/corona.nwk \
  --alignment /home/vmuser/examples/corona.afa \
  --output-node-data /home/vmuser/examples/corona_nt_muts.json \
  --inference joint


Inferred ancestral sequence states using TreeTime:
	Sagulenko et al. TreeTime: Maximum-likelihood phylodynamic analysis
	Virus Evolution, vol 4, https://academic.oup.com/ve/article/4/1/vex042/4794731

ancestral mutations written to /home/vmuser/examples/corona_nt_muts.json


In [17]:
### Identify Amino-Acid Mutations
!augur translate \
  --tree /home/vmuser/examples/corona.nwk \
  --ancestral-sequences /home/vmuser/examples/corona_nt_muts.json \
  --reference-sequence /home/vmuser/examples/NC_045512.2.gb \
  --output /home/vmuser/examples/corona_aa_muts.json

Read in 12 features from reference sequence file
amino acid mutations written to /home/vmuser/examples/corona_aa_muts.json


In [1]:
%%file lat_longs.tsv
country,china,31.0740838,107.7450643
country,usa,42.6584011,-80.2716978
region,china,31.0740838,107.7450643
region,usa,42.6584011,-80.2716978

Overwriting lat_longs.tsv


In [7]:
%%file colors.tsv
country,china,#511EA8
country,usa,#cd5700
region,china,#511EA8
region,usa,#cd5700

Overwriting colors.tsv


In [8]:
!sed -i 's|,|\t|g' lat_longs.tsv

In [9]:
!sed -i 's|,|\t|g' colors.tsv

In [5]:
%%file auspice_config.json
{
  "title": "Covid-19 5 samples",
  "maintainers": [
    {"name": "Your name", "url": "your url"}
  ],
  "build_url": "your build url",
  "colorings": [
    {
      "key": "gt",
      "title": "Genotype",
      "type": "categorical"
    },
    {
      "key": "num_date",
      "title": "Date",
      "type": "continuous"
    },
    {
      "key": "author",
      "title": "Author",
      "type": "categorical"
    },
    {
      "key": "country",
      "title": "Country",
      "type": "categorical"
    },
    {
      "key": "region",
      "title": "Region",
      "type": "categorical"
    }
  ],
  "geo_resolutions": [
    "country",
    "region"
  ],
  "panels": [
     "tree",
     "map"
  ],
  "display_defaults": {
    "map_triplicate": true
  },
  "filters": [
    "country",
    "region",
    "author"
  ]
}

Overwriting auspice_config.json


In [11]:
### Export the Results
!augur export v2 \
  --tree /home/vmuser/examples/corona.nwk \
  --metadata /home/vmuser/examples/metadata.tsv \
  --node-data /home/vmuser/examples/corona_branch_lengths.json \
              /home/vmuser/examples/corona_traits.json \
              /home/vmuser/examples/corona_nt_muts.json \
              /home/vmuser/examples/corona_aa_muts.json \
  --colors /home/vmuser/examples/colors.tsv \
  --lat-longs /home/vmuser/examples/lat_longs.tsv \
  --auspice-config /home/vmuser/examples/auspice_config.json \
  --output /home/vmuser/examples/corona.json

Validating schema of '/home/vmuser/examples/corona_aa_muts.json'...
Validating config file /home/vmuser/examples/auspice_config.json against the JSON schema
Validating schema of '/home/vmuser/examples/auspice_config.json'...
Validating produced JSON
Validating schema of '/home/vmuser/examples/corona.json'...
Validating that the JSON is internally consistent...
Validation of '/home/vmuser/examples/corona.json' succeeded.



## Nextstrain visualization

* Download corona.json file to `/local/path/data`
* Install docker on local PC
* Get docker image: `docker pull nextstrain/base`
* Run docker image: `docker run -it -p4000:4000 -v /local/path:/nextstrain.org -e HOST=0.0.0.0 nextstrain/base auspice view --datasetDir /nextstrain.org/data`
* Access `localhost:4000` in browser